# Lab 6: Object Detection - Part 1
## Bounding Boxes, YOLO, and Visualization

**Duration**: ~3 hours

### Learning Objectives
By the end of this lab, you will be able to:
1. Understand bounding box representations
2. Use a pre-trained YOLO model for object detection
3. Visualize detections with bounding boxes and labels
4. Calculate Intersection over Union (IoU)

### Prerequisites
- Completed Labs 1-3 (Python, NumPy, basic ML)
- Basic understanding of images as arrays

### What is Object Detection?

```
CLASSIFICATION vs DETECTION:

Classification:                Detection:
"What is in the image?"        "What is in the image AND where?"

┌───────────────┐              ┌───────────────┐
│               │              │  ┌─────┐      │
│    🐕         │              │  │ 🐕  │      │
│               │              │  │ dog │      │
│               │              │  └─────┘      │
└───────────────┘              │    ┌────┐     │
                               │    │ 🐈 │     │
Output: "dog"                  │    │cat │     │
                               │    └────┘     │
                               └───────────────┘
                               
                               Output: [(dog, box1), (cat, box2)]
```

In [ ]:
# Install required packages
# !pip install ultralytics supervision opencv-python matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

---

# Part 1: Understanding Bounding Boxes

## What is a Bounding Box?

A bounding box is a rectangle that surrounds an object in an image.

```
IMAGE COORDINATES:

(0,0) ──────────────────► x
  │
  │    ┌─────────────┐
  │    │             │
  │    │   Object    │ height (h)
  │    │             │
  │    └─────────────┘
  │         width (w)
  │
  ▼
  y
```

## Different Box Formats

| Format | Description | Example |
|--------|-------------|----------|
| **xyxy** | (x1, y1, x2, y2) - corners | (100, 50, 200, 150) |
| **xywh** | (x, y, width, height) - top-left + size | (100, 50, 100, 100) |
| **cxcywh** | (cx, cy, width, height) - center + size | (150, 100, 100, 100) |

In [ ]:
# SOLVED EXAMPLE: Box Format Conversions

def xyxy_to_xywh(box):
    """Convert (x1, y1, x2, y2) to (x, y, w, h)"""
    x1, y1, x2, y2 = box
    return (x1, y1, x2 - x1, y2 - y1)

def xywh_to_xyxy(box):
    """Convert (x, y, w, h) to (x1, y1, x2, y2)"""
    x, y, w, h = box
    return (x, y, x + w, y + h)

def xyxy_to_cxcywh(box):
    """Convert (x1, y1, x2, y2) to (cx, cy, w, h)"""
    x1, y1, x2, y2 = box
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2
    cy = y1 + h / 2
    return (cx, cy, w, h)

# Example
box_xyxy = (100, 50, 200, 150)
print(f"xyxy format: {box_xyxy}")
print(f"xywh format: {xyxy_to_xywh(box_xyxy)}")
print(f"cxcywh format: {xyxy_to_cxcywh(box_xyxy)}")

In [ ]:
# SOLVED EXAMPLE: Visualize Box Formats

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Create a simple image
img = np.ones((200, 300, 3)) * 0.9  # Light gray background

box = (100, 50, 200, 150)  # xyxy format
x1, y1, x2, y2 = box

# xyxy format
axes[0].imshow(img)
rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                          linewidth=3, edgecolor='red', facecolor='none')
axes[0].add_patch(rect)
axes[0].plot(x1, y1, 'ro', markersize=10)
axes[0].plot(x2, y2, 'bo', markersize=10)
axes[0].annotate(f'(x1={x1}, y1={y1})', (x1, y1-10), color='red')
axes[0].annotate(f'(x2={x2}, y2={y2})', (x2, y2+15), color='blue')
axes[0].set_title('xyxy: Two Corners')

# xywh format
x, y, w, h = xyxy_to_xywh(box)
axes[1].imshow(img)
rect = patches.Rectangle((x, y), w, h, 
                          linewidth=3, edgecolor='green', facecolor='none')
axes[1].add_patch(rect)
axes[1].plot(x, y, 'go', markersize=10)
axes[1].annotate(f'(x={x}, y={y})', (x, y-10), color='green')
axes[1].annotate(f'w={w}', (x + w/2, y2+15), color='green', ha='center')
axes[1].annotate(f'h={h}', (x2+10, y + h/2), color='green', va='center')
axes[1].set_title('xywh: Top-Left + Size')

# cxcywh format
cx, cy, w, h = xyxy_to_cxcywh(box)
axes[2].imshow(img)
rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                          linewidth=3, edgecolor='purple', facecolor='none')
axes[2].add_patch(rect)
axes[2].plot(cx, cy, 'mo', markersize=10)
axes[2].annotate(f'Center ({cx}, {cy})', (cx+5, cy-5), color='purple')
axes[2].set_title('cxcywh: Center + Size')

for ax in axes:
    ax.set_xlim(0, 300)
    ax.set_ylim(200, 0)

plt.tight_layout()
plt.show()

## Drawing Boxes on Images

In [ ]:
# SOLVED EXAMPLE: Draw Bounding Box with OpenCV

def draw_box(image, box, label, color=(0, 255, 0), thickness=2):
    """
    Draw a bounding box with label on an image.
    
    Args:
        image: numpy array (H, W, 3) in RGB
        box: (x1, y1, x2, y2) format
        label: text to display
        color: RGB tuple
        thickness: line thickness
    """
    img = image.copy()
    x1, y1, x2, y2 = [int(v) for v in box]
    
    # Draw rectangle
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    
    # Draw label background
    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    cv2.rectangle(img, (x1, y1 - text_height - 10), (x1 + text_width + 5, y1), color, -1)
    
    # Draw label text
    cv2.putText(img, label, (x1 + 2, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    return img

# Create a sample image
sample_img = np.ones((400, 600, 3), dtype=np.uint8) * 200

# Draw some boxes
sample_img = draw_box(sample_img, (50, 50, 200, 200), "Dog: 0.95", color=(0, 255, 0))
sample_img = draw_box(sample_img, (300, 100, 500, 350), "Cat: 0.87", color=(255, 0, 0))
sample_img = draw_box(sample_img, (150, 250, 350, 380), "Bird: 0.72", color=(0, 0, 255))

plt.figure(figsize=(10, 7))
plt.imshow(sample_img)
plt.title('Bounding Boxes with Labels')
plt.axis('off')
plt.show()

## Question 1.1

Write a function `cxcywh_to_xyxy(box)` that converts from center format to corner format.

In [ ]:
# YOUR CODE HERE

def cxcywh_to_xyxy(box):
    """Convert (cx, cy, w, h) to (x1, y1, x2, y2)"""
    # Hint: cx = x1 + w/2, so x1 = cx - w/2
    pass

# Test
# box_cxcywh = (150, 100, 100, 100)
# Expected output: (100, 50, 200, 150)

---

# Part 2: Using Pre-trained YOLO

## What is YOLO?

**YOLO = You Only Look Once**

Unlike older methods that look at many image regions, YOLO processes the entire image in one pass.

```
OLDER METHODS:              YOLO:

Look at 1000s of regions    Look at image ONCE
┌─┬─┬─┬─┐                   ┌───────────┐
├─┼─┼─┼─┤  → Slow           │           │ → Fast!
├─┼─┼─┼─┤                   │   Grid    │
├─┼─┼─┼─┤                   │           │
└─┴─┴─┴─┘                   └───────────┘
```

We'll use **YOLOv8** from Ultralytics - the latest and easiest to use!

In [ ]:
from ultralytics import YOLO

# Load pre-trained model (downloads automatically)
model = YOLO('yolov8n.pt')  # 'n' = nano (smallest, fastest)

print("Model loaded!")
print(f"Model type: YOLOv8 Nano")
print(f"Number of classes: {len(model.names)}")
print(f"\nFirst 20 classes: {list(model.names.values())[:20]}")

In [ ]:
# SOLVED EXAMPLE: Download a Sample Image

import urllib.request

# Download a sample image
url = "https://ultralytics.com/images/bus.jpg"
image_path = "bus.jpg"

if not os.path.exists(image_path):
    urllib.request.urlretrieve(url, image_path)
    print(f"Downloaded {image_path}")

# Display image
img = Image.open(image_path)
plt.figure(figsize=(12, 8))
plt.imshow(img)
plt.title('Sample Image')
plt.axis('off')
plt.show()

print(f"Image size: {img.size}")

In [ ]:
# SOLVED EXAMPLE: Run Detection

# Run inference
results = model(image_path)

# Get the first result (we only have one image)
result = results[0]

print("Detection Results:")
print(f"  Number of detections: {len(result.boxes)}")
print(f"  Inference time: {result.speed['inference']:.1f} ms")

In [ ]:
# SOLVED EXAMPLE: Understanding the Output

# Access detection boxes
boxes = result.boxes

print("\nDetailed Results:")
print("=" * 60)

for i, box in enumerate(boxes):
    # Get coordinates (xyxy format)
    x1, y1, x2, y2 = box.xyxy[0].tolist()
    
    # Get confidence
    conf = box.conf[0].item()
    
    # Get class
    cls_id = int(box.cls[0].item())
    cls_name = model.names[cls_id]
    
    print(f"Detection {i+1}:")
    print(f"  Class: {cls_name}")
    print(f"  Confidence: {conf:.2%}")
    print(f"  Box (xyxy): ({x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f})")
    print()

In [ ]:
# SOLVED EXAMPLE: Visualize Detections (Built-in)

# YOLO has built-in visualization
annotated_image = result.plot()

# Convert BGR to RGB for matplotlib
annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(14, 10))
plt.imshow(annotated_image)
plt.title('YOLO Detections')
plt.axis('off')
plt.show()

## Using Different Model Sizes

In [ ]:
# SOLVED EXAMPLE: Compare Model Sizes

model_sizes = {
    'yolov8n.pt': 'Nano (fastest)',
    'yolov8s.pt': 'Small',
    'yolov8m.pt': 'Medium',
}

print("Model Size Comparison:")
print("=" * 50)

for model_name, description in model_sizes.items():
    temp_model = YOLO(model_name)
    results = temp_model(image_path, verbose=False)
    
    inference_time = results[0].speed['inference']
    num_detections = len(results[0].boxes)
    
    print(f"{description:20s}: {inference_time:6.1f} ms, {num_detections} detections")

print("\nNote: Larger models are more accurate but slower.")

## Question 2.1

Download your own image (or use one from your computer) and run YOLO detection on it.
Print all detected objects with their confidence scores.

In [ ]:
# YOUR CODE HERE

# Option 1: Use a URL
# your_image_url = "https://example.com/image.jpg"

# Option 2: Upload an image to Colab
# from google.colab import files
# uploaded = files.upload()

# Run detection
# ...

---

# Part 3: Custom Visualization with Supervision

The `supervision` library provides beautiful, customizable visualizations.

In [ ]:
import supervision as sv

print(f"Supervision version: {sv.__version__}")

In [ ]:
# SOLVED EXAMPLE: Supervision Visualization

# Get YOLO results
results = model(image_path)
result = results[0]

# Convert to supervision format
detections = sv.Detections.from_ultralytics(result)

print(f"Number of detections: {len(detections)}")
print(f"\nDetection attributes:")
print(f"  xyxy: {detections.xyxy.shape}")
print(f"  confidence: {detections.confidence}")
print(f"  class_id: {detections.class_id}")

In [ ]:
# SOLVED EXAMPLE: Custom Annotator

# Load image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Create annotators
box_annotator = sv.BoxAnnotator(
    thickness=2,
    color=sv.ColorPalette.DEFAULT
)

label_annotator = sv.LabelAnnotator(
    text_scale=0.5,
    text_padding=5
)

# Generate labels
labels = [
    f"{model.names[class_id]} {confidence:.2f}"
    for class_id, confidence in zip(detections.class_id, detections.confidence)
]

# Annotate
annotated = box_annotator.annotate(scene=image.copy(), detections=detections)
annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

plt.figure(figsize=(14, 10))
plt.imshow(annotated)
plt.title('Custom Visualization with Supervision')
plt.axis('off')
plt.show()

In [ ]:
# SOLVED EXAMPLE: Filter by Confidence

# Only keep high confidence detections
high_conf = detections[detections.confidence > 0.5]

print(f"Original detections: {len(detections)}")
print(f"After filtering (conf > 0.5): {len(high_conf)}")

# Visualize
labels = [
    f"{model.names[class_id]} {confidence:.2f}"
    for class_id, confidence in zip(high_conf.class_id, high_conf.confidence)
]

annotated = box_annotator.annotate(scene=image.copy(), detections=high_conf)
annotated = label_annotator.annotate(scene=annotated, detections=high_conf, labels=labels)

plt.figure(figsize=(14, 10))
plt.imshow(annotated)
plt.title('High Confidence Detections Only (> 0.5)')
plt.axis('off')
plt.show()

In [ ]:
# SOLVED EXAMPLE: Filter by Class

# Only keep 'person' detections
person_class_id = list(model.names.keys())[list(model.names.values()).index('person')]
person_detections = detections[detections.class_id == person_class_id]

print(f"Person class ID: {person_class_id}")
print(f"Number of persons detected: {len(person_detections)}")

# Visualize
labels = [f"person {conf:.2f}" for conf in person_detections.confidence]

person_annotator = sv.BoxAnnotator(thickness=3, color=sv.Color.RED)
annotated = person_annotator.annotate(scene=image.copy(), detections=person_detections)
annotated = label_annotator.annotate(scene=annotated, detections=person_detections, labels=labels)

plt.figure(figsize=(14, 10))
plt.imshow(annotated)
plt.title('Only Persons')
plt.axis('off')
plt.show()

## Question 3.1

Create a visualization that:
1. Shows only vehicles (car, truck, bus)
2. Uses a different color for each vehicle type
3. Displays the count of each vehicle type in the title

In [ ]:
# YOUR CODE HERE

# Vehicle class IDs (you can find these by printing model.names)
# car = 2, truck = 7, bus = 5


---

# Part 4: Intersection over Union (IoU)

## What is IoU?

IoU measures how much two boxes overlap.

```
IoU = Area of Intersection / Area of Union

┌───────────┐
│   Box A   │
│     ┌─────┼───────┐
│     │XXXXX│       │
└─────┼─────┘       │
      │    Box B    │
      └─────────────┘

XXX = Intersection

IoU = Area(XXX) / Area(A ∪ B)

IoU = 0: No overlap
IoU = 1: Perfect overlap
IoU > 0.5: Usually considered "good match"
```

In [ ]:
# SOLVED EXAMPLE: Calculate IoU

def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union between two boxes.
    
    Args:
        box1, box2: Boxes in (x1, y1, x2, y2) format
    
    Returns:
        IoU value (0 to 1)
    """
    # Unpack coordinates
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Calculate intersection coordinates
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    # Calculate intersection area
    if x2_i < x1_i or y2_i < y1_i:
        intersection = 0  # No overlap
    else:
        intersection = (x2_i - x1_i) * (y2_i - y1_i)
    
    # Calculate areas of both boxes
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Calculate union
    union = area1 + area2 - intersection
    
    # Calculate IoU
    iou = intersection / union if union > 0 else 0
    
    return iou

# Test cases
box_a = (0, 0, 100, 100)
box_b = (50, 50, 150, 150)
box_c = (200, 200, 300, 300)  # No overlap with A
box_d = (0, 0, 100, 100)  # Same as A

print(f"IoU(A, B) = {calculate_iou(box_a, box_b):.4f}")
print(f"IoU(A, C) = {calculate_iou(box_a, box_c):.4f} (no overlap)")
print(f"IoU(A, D) = {calculate_iou(box_a, box_d):.4f} (perfect overlap)")

In [ ]:
# SOLVED EXAMPLE: Visualize IoU

def visualize_iou(box1, box2):
    """Visualize two boxes and their IoU."""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Draw boxes
    rect1 = patches.Rectangle(
        (box1[0], box1[1]), box1[2]-box1[0], box1[3]-box1[1],
        linewidth=2, edgecolor='blue', facecolor='blue', alpha=0.3,
        label='Box A'
    )
    rect2 = patches.Rectangle(
        (box2[0], box2[1]), box2[2]-box2[0], box2[3]-box2[1],
        linewidth=2, edgecolor='red', facecolor='red', alpha=0.3,
        label='Box B'
    )
    
    ax.add_patch(rect1)
    ax.add_patch(rect2)
    
    # Calculate and display IoU
    iou = calculate_iou(box1, box2)
    
    ax.set_xlim(-50, 350)
    ax.set_ylim(350, -50)
    ax.set_title(f'IoU = {iou:.4f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    return fig

# Show different overlap cases
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

cases = [
    ((0, 0, 100, 100), (50, 50, 150, 150), "Partial Overlap"),
    ((0, 0, 100, 100), (200, 200, 300, 300), "No Overlap"),
    ((0, 0, 100, 100), (25, 25, 125, 125), "High Overlap"),
]

for ax, (box1, box2, title) in zip(axes, cases):
    rect1 = patches.Rectangle(
        (box1[0], box1[1]), box1[2]-box1[0], box1[3]-box1[1],
        linewidth=2, edgecolor='blue', facecolor='blue', alpha=0.3
    )
    rect2 = patches.Rectangle(
        (box2[0], box2[1]), box2[2]-box2[0], box2[3]-box2[1],
        linewidth=2, edgecolor='red', facecolor='red', alpha=0.3
    )
    
    ax.add_patch(rect1)
    ax.add_patch(rect2)
    
    iou = calculate_iou(box1, box2)
    ax.set_title(f'{title}\nIoU = {iou:.3f}')
    ax.set_xlim(-50, 350)
    ax.set_ylim(350, -50)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## Why IoU Matters

IoU is used everywhere in object detection:

1. **Training**: Match predicted boxes to ground truth
2. **NMS**: Remove duplicate detections (covered in Lab 7)
3. **Evaluation**: Measure model performance (mAP)

In [ ]:
# SOLVED EXAMPLE: When is a Detection "Correct"?

# Ground truth (what's actually in the image)
ground_truth = (100, 100, 300, 300)

# Different predictions
predictions = [
    (105, 95, 305, 305),   # Good prediction
    (150, 150, 350, 350),  # Okay prediction
    (250, 250, 400, 400),  # Poor prediction
    (400, 400, 500, 500),  # Wrong prediction
]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, pred in zip(axes, predictions):
    # Ground truth (green)
    gt_rect = patches.Rectangle(
        (ground_truth[0], ground_truth[1]), 
        ground_truth[2]-ground_truth[0], ground_truth[3]-ground_truth[1],
        linewidth=2, edgecolor='green', facecolor='green', alpha=0.3,
        label='Ground Truth'
    )
    
    # Prediction (blue)
    pred_rect = patches.Rectangle(
        (pred[0], pred[1]), pred[2]-pred[0], pred[3]-pred[1],
        linewidth=2, edgecolor='blue', facecolor='blue', alpha=0.3,
        label='Prediction'
    )
    
    ax.add_patch(gt_rect)
    ax.add_patch(pred_rect)
    
    iou = calculate_iou(ground_truth, pred)
    status = "CORRECT" if iou >= 0.5 else "WRONG"
    color = 'green' if iou >= 0.5 else 'red'
    
    ax.set_title(f'IoU = {iou:.3f}\n{status}', color=color, fontweight='bold')
    ax.set_xlim(0, 550)
    ax.set_ylim(550, 0)
    ax.set_aspect('equal')

plt.suptitle('Detection is CORRECT if IoU >= 0.5', fontsize=14)
plt.tight_layout()
plt.show()

## Question 4.1

Given these two boxes, calculate the IoU by hand first, then verify with code:
- Box A: (10, 10, 50, 50)
- Box B: (30, 30, 70, 70)

Hint: Draw it on paper first!

In [ ]:
# YOUR CODE HERE

# Hand calculation:
# Area A = ?
# Area B = ?
# Intersection area = ?
# Union area = ?
# IoU = ?

# Verify with code


---

# Challenge Problems

## Challenge 1: Process a Video

Run YOLO on a video file and count how many times each object class appears across all frames.

In [ ]:
# YOUR CODE HERE

# Download a sample video
# video_url = "https://..."

# Process each frame
# ...

## Challenge 2: Object Counting App

Create a function that takes an image and returns a dictionary counting each detected object type.

In [ ]:
# YOUR CODE HERE

def count_objects(image_path, confidence_threshold=0.5):
    """
    Count objects in an image.
    
    Returns:
        dict: {class_name: count}
    """
    pass

# Test
# counts = count_objects("bus.jpg")
# print(counts)  # Expected: {'person': 4, 'bus': 1, ...}

## Challenge 3: IoU Matrix

Given a list of predicted boxes and ground truth boxes, create an IoU matrix showing the IoU between every pair.

In [ ]:
# YOUR CODE HERE

def iou_matrix(predictions, ground_truths):
    """
    Calculate IoU between all pairs of boxes.
    
    Returns:
        numpy array of shape (len(predictions), len(ground_truths))
    """
    pass

# Test
# preds = [(0, 0, 50, 50), (100, 100, 150, 150)]
# gts = [(5, 5, 55, 55), (90, 90, 140, 140), (200, 200, 250, 250)]
# matrix = iou_matrix(preds, gts)
# print(matrix.shape)  # (2, 3)

---

# Summary

## What We Learned

| Concept | Description |
|---------|-------------|
| **Bounding Box** | Rectangle around an object: (x1, y1, x2, y2) |
| **YOLO** | Fast object detector - "You Only Look Once" |
| **Confidence** | How sure the model is (0 to 1) |
| **IoU** | Intersection over Union - measures box overlap |
| **Supervision** | Library for beautiful detection visualization |

## Key Code Patterns

```python
# Load YOLO
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

# Run detection
results = model(image_path)
boxes = results[0].boxes

# Visualize with supervision
import supervision as sv
detections = sv.Detections.from_ultralytics(results[0])
annotator = sv.BoxAnnotator()
annotated = annotator.annotate(image, detections)

# Calculate IoU
iou = calculate_iou(box1, box2)
```

## What's Next?

In **Lab 7**, we'll learn about:
- Non-Maximum Suppression (NMS)
- Fine-tuning YOLO on custom data
- Evaluation metrics (mAP)